In [23]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [24]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import sum,count
from pyspark.sql.functions import stddev_pop
from pyspark.sql.functions import max
from pyspark.sql.functions import mean ,corr,max, min,stddev_pop ,stddev_samp,covar_pop,covar_samp
spark = SparkSession.builder.getOrCreate()
sc =SparkContext.getOrCreate()


In [25]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
order_items = [(1,2,3,5,49000),
                 (2,2,2,5,49000),
                 (3,3,3,5,49000),
                 (4,4,2,6,49000)]

In [26]:
schema = StructType([
    StructField("order_id",IntegerType(),True),
    StructField("item_id",StringType(),True),
    StructField("product_id",StringType(),True),
    StructField("quantity",StringType(),True),
    StructField("unit_price",StringType(),True),
])

In [27]:
order_items = spark.createDataFrame(data = order_items,schema = schema)

In [28]:
order_items.printSchema()
order_items.show(truncate=False)

root
 |-- order_id: integer (nullable = true)
 |-- item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- unit_price: string (nullable = true)

+--------+-------+----------+--------+----------+
|order_id|item_id|product_id|quantity|unit_price|
+--------+-------+----------+--------+----------+
|1       |2      |3         |5       |49000     |
|2       |2      |2         |5       |49000     |
|3       |3      |3         |5       |49000     |
|4       |4      |2         |6       |49000     |
+--------+-------+----------+--------+----------+



In [29]:
Orders_data = [(1,8,"in progress",2,"2021"),
             (2,9,"pending",3,"2021"),
             (3,11,"arrived",2,"2021"),
             (4,12,"pending",5,"2021"),
             ]

In [30]:
schema = StructType([
    StructField("order_id",IntegerType(),True),
    StructField("customer_id",StringType(),True),
    StructField("status",StringType(),True),
    StructField("salesman_id",IntegerType(),True),
    StructField("order_date",StringType(),True),
    
])

In [31]:
Orders_data=spark.createDataFrame(data = Order1_data,schema=schema)
Orders_data.printSchema()
Orders_data.show(truncate = False)

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- salesman_id: integer (nullable = true)
 |-- order_date: string (nullable = true)

+--------+-----------+-----------+-----------+----------+
|order_id|customer_id|status     |salesman_id|order_date|
+--------+-----------+-----------+-----------+----------+
|1       |8          |in progress|2          |2021      |
|2       |9          |pending    |3          |2021      |
|3       |11         |arrived    |2          |2021      |
|4       |12         |pending    |5          |2021      |
+--------+-----------+-----------+-----------+----------+



In [32]:
Orders_data.createOrReplaceTempView("Orders_data")
order_items.createOrReplaceTempView("order_items")

In [46]:
spark.sql("select year(order_date) as year,sum(quantity*unit_price) as revenue from order_items i NATURAL JOIN Orders_data d  group by year(order_date) order by year").explain()

== Physical Plan ==
*(7) Sort [year#374 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(year#374 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#581]
   +- *(6) HashAggregate(keys=[year(cast(order_date#177 as date))#380], functions=[sum((cast(quantity#145 as double) * cast(unit_price#146 as double)))])
      +- Exchange hashpartitioning(year(cast(order_date#177 as date))#380, 200), ENSURE_REQUIREMENTS, [id=#577]
         +- *(5) HashAggregate(keys=[year(cast(order_date#177 as date)) AS year(cast(order_date#177 as date))#380], functions=[partial_sum((cast(quantity#145 as double) * cast(unit_price#146 as double)))])
            +- *(5) Project [quantity#145, unit_price#146, order_date#177]
               +- *(5) SortMergeJoin [order_id#142], [order_id#173], Inner
                  :- *(2) Sort [order_id#142 ASC NULLS FIRST], false, 0
                  :  +- Exchange hashpartitioning(order_id#142, 200), ENSURE_REQUIREMENTS, [id=#562]
                  :     +- *(1) Project [order